In [23]:
import urllib
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os


In [24]:
def load_sgindeed_jobs(i):
    url = ('https://sg.indeed.com/jobs?l=singapore&start='+str(i))
    page = requests.get(url)
    soup = BeautifulSoup(page.content,"html.parser")
    print(soup)
    job_soup = soup.find(id = "resultsCol")

    return job_soup

In [25]:
def extract_job_title(job_elem):
    title_elem = job_elem.find('h2',class_='jobTitle')
    title = title_elem.text.strip()
    return title

In [26]:
def extract_link(job_elem):
    links = job_elem
    if links:
        link = links['href']
        link = 'sg.indeed.com'+link
    else:
        link = ""
    return link

In [27]:
def extract_date(job_elem):
    date_elem = job_elem.find('span',class_ = 'date')
    date = date_elem.text.strip()
    return date


In [28]:
def extract_description(job_elem):
    description_elem = job_elem.find('div',class_='job-snippet')
    description = description_elem.text.strip()
    return description    

In [29]:
def extract_salary(job_elem):
    salary_elem = job_elem.find('div',class_='salary-snippet-container')
    if salary_elem:
        salary = salary_elem.findChildren('span')
        salary = salary_elem.text.strip()
    else:
        salary = ""
    return salary

In [30]:
def extract_company_name(job_elem):
    company_elem = job_elem.find('span',class_='companyName')
    if company_elem:
        company = company_elem.text.strip()
    else:
        company = ""
    return company

In [31]:
def find_jobs_from(filename = "job_listings.xls"):
    
    jobs_df = pd.DataFrame({'title':[],
                        'company name':[],
                        'link':[],
                        'date listed':[],
                        'salary':[],
                        'job description':[]})


    for i in range(10,201,10):
        job_soup = load_sgindeed_jobs(i)
   
        job_subdf = extract_job_information_indeed(job_soup)
        jobs_df = jobs_df.append(job_subdf,ignore_index=True)

    print("{} listings found online".format(len(jobs_df)))
    jobs_df = jobs_df.dropna(())
    
    jobs_df.to_excel(filename)
    print('After removing incomplete postings, {} new job postings retrieved. Stored in {}.'.format(len(jobs_df),filename))


In [32]:
def extract_job_information_indeed(job_soup):
    job_elems = job_soup.find_all('div',class_='slider_container')
    #print("job elems length ", len(job_elems))


    titles = []
    for job_elem in job_elems:
        titles.append(extract_job_title(job_elem))
    
    companies = []
    for job_elem in job_elems:
        companies.append(extract_company_name(job_elem))
  
    links = []
    for job_elem in job_elems:
        job_elem = job_elem.find_parent('a')
        links.append(extract_link(job_elem))
  

    dates = []
    for job_elem in job_elems:
        dates.append(extract_date(job_elem))

    salaries = []
    for job_elem in job_elems:
        salaries.append(extract_salary(job_elem))

    job_desc = []
    for job_elem in job_elems:
        job_desc.append(extract_description(job_elem))
    
    jobs_df = pd.DataFrame({'title':titles,
                        'company name': companies,
                        'link':links,
                        'date listed':dates,
                        'salary':salaries,
                        'job description': job_desc})
    
    return jobs_df

    

In [33]:

find_jobs_from()

<Response [200]>


AttributeError: 'NoneType' object has no attribute 'find_all'